<a href="https://colab.research.google.com/github/fatimakha202-cmyk/Cats-Vs-Dogs-Image-Classification/blob/master/2cats_vs_dogs_image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cat Dog Classification
===

In [7]:
# ------------------------------
# Step 1: Install necessary libraries
# ------------------------------
!pip install tensorflow matplotlib numpy opencv-python

# ------------------------------
# Step 2: Import dependencies
# ------------------------------
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

# ------------------------------
# Step 3: Verify imports
# ------------------------------
print("All dependencies installed and imported successfully!")
print("TensorFlow version:", tf.__version__)


All dependencies installed and imported successfully!
TensorFlow version: 2.19.0


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
!git clone https://github.com/fatimakha202-cmyk/Cats-Vs-Dogs-Image-Classification.git


fatal: destination path 'Cats-Vs-Dogs-Image-Classification' already exists and is not an empty directory.


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Define some mathematical functions

In [11]:
# Here 0 means Cat and 1 means Dog
CAT = 0
DOG = 1
#Returns whether the low memory mode is used.
IS_LOW_MEMORY_MODE = True
#current working directory of a process.
cwd = os.getcwd()
#This method is called when RandomState is initialized
np.random.seed(2124)

In [12]:
#Method to prepare a file
def prepare_file():
  file_list = ['train','test']
  flag = True

  # Path to the data directory within the cloned repository
  repo_data_path = os.path.join(cwd, 'Cats-Vs-Dogs-Image-Classification', 'data')

  # Ensure the target extraction directory exists
  target_data_dir = os.path.join(cwd, 'data')
  os.makedirs(target_data_dir, exist_ok=True)

  for filename_prefix in file_list:
    zip_filename = filename_prefix + '.zip'
    zip_filepath = os.path.join(repo_data_path, zip_filename)

    # Path to extract images for current file_list item (e.g., 'train' or 'test')
    extract_path = os.path.join(target_data_dir, filename_prefix)
    os.makedirs(extract_path, exist_ok=True)

    print(f"Extracting {zip_filepath} to {extract_path}")
    with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
      zip_ref.extractall(extract_path)

  return flag

In [13]:
#Method to read the image label
def read_image_label_list(folder_dir):
    dir_list = os.listdir(os.path.join(cwd,folder_dir))
    filenames = []
    labels = []

    for i, d in enumerate(dir_list):
        if re.search("train",folder_dir):
            if re.search("cat", d):
                labels.append(CAT)
            else:
                labels.append(DOG)
        else:
            labels.append(-1)
        filenames.append(os.path.join(cwd, folder_dir, d))
    return filenames, labels

In [14]:
#Method to read the image from disk
def read_images_from_disk(input_queue):
    filename = input_queue[0]
    label = input_queue[1]

    file_contents = tf.read_file(filename)
    image = tf.image.decode_image(file_contents, channels=3)
    image.set_shape([None, None, 3])

    return image, label

In [15]:
#Method to generate input function
def gen_input_fn(image_list, label_list, batch_size, shuffle):

    def input_fn():
        images = tf.convert_to_tensor(image_list, dtype=tf.string)
        labels = tf.convert_to_tensor(label_list, dtype=tf.int32)

        input_queue = tf.train.slice_input_producer(
            [images, labels],
            capacity=batch_size * 5,
            shuffle=shuffle,
            name="file_input_queue"
        )

        image, label = read_images_from_disk(input_queue)
        image = tf.image.resize_images(image, (224, 224), tf.image.ResizeMethod.NEAREST_NEIGHBOR)

        image_batch, label_batch = tf.train.batch(
            [image, label],
            batch_size=batch_size,
            num_threads=1,
            name="batch_queue",
            capacity=batch_size * 10,
            allow_smaller_final_batch = False
        )

        return (
            tf.identity(image_batch, name="features"),
            tf.identity(label_batch, name="label")
        )

    return input_fn

In [16]:
#Method to train a valid input function
def train_valid_input_fn(data_dir, train_batch_size, valid_batch_size=None):
    img, labels = read_image_label_list(data_dir)
    img = np.array(img)
    labels = np.array(labels)
    data_size = img.shape[0]

    print("Data size: " + str(data_size))
    split = int(0.7 * data_size)

    random_seq = np.random.permutation(data_size)

    img = img[random_seq]
    labels = labels[random_seq]

    if valid_batch_size == None:
        valid_batch_size = train_batch_size

    return (
        gen_input_fn(img[0:split], labels[0:split], train_batch_size, shuffle = True),
        gen_input_fn(img[split:], labels[split:], valid_batch_size, shuffle = False)
           )

In [17]:
#Method to test input function
def test_input_fn(data_dir,batch_size):
    image_list, label_list = read_image_label_list(data_dir)
    return gen_input_fn(image_list, label_list, batch_size, shuffle = False), image_list

### Data visualize

In [18]:
#Method to plot data
def plot_img(data, label=None):
    plt.ion()
    plt.figure()
    plt.imshow(data)
    if label is not None:
        plt.title(label)

In [19]:
import tensorflow as tf

def preview_img_tf2(batch_size=5):
    import matplotlib.pyplot as plt
    import os
    import cv2

    path = '/content/data/train'
    classes = os.listdir(path)
    images = []
    labels = []

    for label, cls in enumerate(classes):
        cls_path = os.path.join(path, cls)
        for img_name in os.listdir(cls_path)[:batch_size]:
            img_path = os.path.join(cls_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (150,150))
            images.append(img)
            labels.append(label)

    for i in range(len(images)):
        plt.imshow(images[i])
        plt.title(str(labels[i]))
        plt.axis('off')
        plt.show()

preview_img_tf2()

In [20]:
if prepare_file():
    print("Files extracted successfully!")

Extracting /content/Cats-Vs-Dogs-Image-Classification/data/train.zip to /content/data/train


NameError: name 'zipfile' is not defined

### Define Model

In [ ]:
#Cat-Dog Method Declaration
def catdog_model(inputs, is_training):
    with tf.variable_scope('catdog', values=[inputs]):
        with slim.arg_scope(
            [slim.conv2d, slim.fully_connected],
            activation_fn=tf.nn.relu6,
            weights_initializer=tf.truncated_normal_initializer(0.0, 0.01)):

            net = inputs

            if IS_LOW_MEMORY_MODE == False:
                net = slim.repeat(net, 2, slim.conv2d, 64, [3, 3], scope='conv1')
                net = slim.max_pool2d(net, [2, 2], scope='pool1')

                net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
                net = slim.max_pool2d(net, [2, 2], scope='pool2')

                net = slim.repeat(net, 4, slim.conv2d, 256, [3, 3], scope='conv3')
                net = slim.max_pool2d(net, [2, 2], scope='pool3')
                net = slim.repeat(net, 4, slim.conv2d, 512, [3, 3], scope='conv4')
                net = slim.max_pool2d(net, [2, 2], scope='pool4')
                net = slim.repeat(net, 4, slim.conv2d, 512, [3, 3], scope='conv5')
                net = slim.max_pool2d(net, [2, 2], scope='pool5')

                net = tf.reshape(net, [-1, 7 * 7 * 512])

                net = slim.fully_connected(net, 2048, scope='fc6')
                net = slim.dropout(net, 0.5, is_training=is_training, scope='dropout6')

                net = slim.fully_connected(net, 2048, scope='fc7')
                net = slim.dropout(net, 0.5, is_training=is_training, scope='dropout7')

                net = slim.fully_connected(net, 2, activation_fn=None, scope='fc8')

            else:
                # Model for my Mac T_T
                net = tf.image.resize_images(net, (72, 72), tf.image.ResizeMethod.NEAREST_NEIGHBOR)

                net = slim.repeat(net, 1, slim.conv2d, 64, [3, 3], scope='conv1')
                net = slim.max_pool2d(net, [2, 2], scope='pool1')

                net = slim.repeat(net, 1, slim.conv2d, 128, [3, 3], scope='conv2')
                net = slim.max_pool2d(net, [2, 2], scope='pool2')

                net = slim.repeat(net, 2, slim.conv2d, 256, [3, 3], scope='conv3')
                net = slim.max_pool2d(net, [2, 2], scope='pool3')

                net = tf.reshape(net, [-1, 9 * 9 * 256])

                net = slim.fully_connected(net, 1024, scope='fc4')
                net = slim.dropout(net, 0.5, is_training=is_training, scope='dropout4')

                net = slim.fully_connected(net, 1024, scope='fc5')
                net = slim.dropout(net, 0.5, is_training=is_training, scope='dropout5')

                net = slim.fully_connected(net, 2, activation_fn=None, scope='fc6')

            return net

In [ ]:
#Cat-Dog Model function
def catdog_model_fn(features, labels, mode, params):

    is_training = False
    if mode == learn.ModeKeys.TRAIN:
        is_training = True

    output = catdog_model(features, is_training)

    log_loss = None
    train_op = None
    eval_metric_ops = None

    softmax_predictions = tf.nn.softmax(output)

    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(
            tf.cast(labels, tf.int32),
            depth = 2
        )
        log_loss = tf.identity(
            tf.losses.log_loss(
                onehot_labels,
                tf.nn.softmax(output),
                reduction = tf.losses.Reduction.MEAN
            ),
            name = "log_loss_tensor"
        )
        eval_metric_ops = {
            "log_loss": log_loss
        }

    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss = log_loss,
            global_step = tf.contrib.framework.get_global_step(),
            learning_rate = params['learning_rate'],
            optimizer = "Adam"
        )

    predictions = {
        'predict': softmax_predictions
    }

    return model_fn.ModelFnOps(
        mode = mode,
        predictions = predictions,
        loss = log_loss,
        train_op = train_op,
        eval_metric_ops = eval_metric_ops
    )

In [31]:
#Feature Engineering function
def feature_engineering_fn(features, labels):
    features = tf.to_float(features)
    features = tf.map_fn(tf.image.per_image_standardization, features)
    return features, labels

tf.logging.set_verbosity(tf.logging.ERROR)

model_path = 'model' if IS_LOW_MEMORY_MODE else 'model'
classifier = learn.Estimator(
    model_fn = catdog_model_fn,
    model_dir = model_path,
    config = run_config(
        save_summary_steps = 10,
        keep_checkpoint_max = 3,
        save_checkpoints_steps = 75
    ),
    feature_engineering_fn = feature_engineering_fn,
    params = {
        'learning_rate': 0.01
    }
)

# Correct path to the extracted training data
train_input_fn, validate_input_fn = train_valid_input_fn('data/train', 32, 64)

logging_hook = tf.train.LoggingTensorHook(
    tensors = {
        'log_loss': 'log_loss_tensor'
    },
    every_n_iter = 3
)

validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    input_fn = validate_input_fn,
    eval_steps = 30,
    every_n_steps = 100,
    name = 'Validatation'
)

AttributeError: module 'tensorflow' has no attribute 'logging'

#### Train the Model

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)
classifier.fit(
    input_fn = train_input_fn,
    steps = 100,
    monitors = [logging_hook, validation_monitor]
)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from model/model.ckpt-200
INFO:tensorflow:Saving checkpoints for 201 into model/model.ckpt.
INFO:tensorflow:log_loss = 6.5479765
INFO:tensorflow:Starting evaluation at 2018-02-06-15:59:45
INFO:tensorflow:Restoring parameters from model/model.ckpt-201
INFO:tensorflow:Evaluation [1/30]
INFO:tensorflow:Evaluation [2/30]
INFO:tensorflow:Evaluation [3/30]
INFO:tensorflow:Evaluation [4/30]
INFO:tensorflow:Evaluation [5/30]
INFO:tensorflow:Evaluation [6/30]
INFO:tensorflow:Evaluation [7/30]
INFO:tensorflow:Evaluation [8/30]
INFO:tensorflow:Evaluation [9/30]
INFO:tensorflow:Evaluation [10/30]
INFO:tensorflow:Evaluation [11/30]
INFO:tensorflow:Evaluation [12/30]
INFO:tensorflow:Evaluation [13/30]
INFO:tensorflow:Evaluation [14/30]
INFO:tensorflow:Evaluation [15/30]
INFO:tensorflow:Evaluation [16/30]
INFO:tensorflow:Evaluation [17/30]
INFO:tensorflow:Evaluation [18/30]
INFO:tensorflow:Evaluation [19/30]
INFO:tensorf

Estimator(params={'learning_rate': 0.01})

In [ ]:
#Model Evaluation
classifier.evaluate(
    input_fn = validate_input_fn,
    steps = 75
)

INFO:tensorflow:Starting evaluation at 2018-02-06-16:08:01
INFO:tensorflow:Restoring parameters from model/model.ckpt-300
INFO:tensorflow:Evaluation [1/75]
INFO:tensorflow:Evaluation [2/75]
INFO:tensorflow:Evaluation [3/75]
INFO:tensorflow:Evaluation [4/75]
INFO:tensorflow:Evaluation [5/75]
INFO:tensorflow:Evaluation [6/75]
INFO:tensorflow:Evaluation [7/75]
INFO:tensorflow:Evaluation [8/75]
INFO:tensorflow:Evaluation [9/75]
INFO:tensorflow:Evaluation [10/75]
INFO:tensorflow:Evaluation [11/75]
INFO:tensorflow:Evaluation [12/75]
INFO:tensorflow:Evaluation [13/75]
INFO:tensorflow:Evaluation [14/75]
INFO:tensorflow:Evaluation [15/75]
INFO:tensorflow:Evaluation [16/75]
INFO:tensorflow:Evaluation [17/75]
INFO:tensorflow:Evaluation [18/75]
INFO:tensorflow:Evaluation [19/75]
INFO:tensorflow:Evaluation [20/75]
INFO:tensorflow:Evaluation [21/75]
INFO:tensorflow:Evaluation [22/75]
INFO:tensorflow:Evaluation [23/75]
INFO:tensorflow:Evaluation [24/75]
INFO:tensorflow:Evaluation [25/75]
INFO:tensorf

{'global_step': 300, 'log_loss': 9.066431, 'loss': 8.163145}

#### Final Prediction

In [32]:
#Print the result
# Correct path to the extracted test data
test_fn, image_test_list = test_input_fn('data/test',32)
test_n = len(image_test_list)

print("Test size: %d" % test_n)

result_file = open(os.path.join(cwd, 'result/result.txt'),'w+')
result_file.write('id,label\n')

predictions = classifier.predict(input_fn = test_fn, as_iterable=True)
for i, p in enumerate(predictions):
    if i >= test_n:
        break

    id = image_test_list[i].split("/")[-1]
    id = id.split(".")[0]

    if i % 100 == 0:
        print("Predict %d %s: %f" % (i,image_test_list[i],p["predict"][1]))

    result_file.write("%s,%f\n" % (id, p["predict"][1]))

result_file.flush()
result_file.close()
print('Finish!!')

FileNotFoundError: [Errno 2] No such file or directory: '/content/data/test'